# Parallelization Lab

In this lab, you will be leveraging several concepts you have learned to obtain a list of links from a web page and crawl and index the pages referenced by those links - both sequentially and in parallel. Follow the steps below to complete the lab.

## Step 1: Use the requests library to retrieve the content from the URL below.

In [1]:
import requests

url = 'https://en.wikipedia.org/wiki/Data_science'

In [2]:
html = requests.get(url).content

## Step 2: Use BeautifulSoup to extract a list of all the unique links on the page.

In [3]:
from bs4 import BeautifulSoup

In [4]:
#Definicimos un objeto en Beautiful soup
soup = BeautifulSoup(html, 'html.parser')
#Buscamos todos los links
links = soup.find_all('a')
#Hacemos get en cada href en todos los links..
links_2 = [i.get('href') for i in links]

links_3 = list(set([str(i) for i in links_2]))
print(links_3)

['#cite_ref-cfjwu01_11-0', 'https://hbr.org/2012/10/data-scientist-the-sexiest-job-of-the-21st-century/', 'https://www.springer.com/book/9784431702085', '/wiki/Special:BookSources/978-0-9825442-0-4', '/wiki/C.F._Jeff_Wu', '/wiki/Multilayer_perceptron', '/wiki/Industry', '/wiki/NYU_Stern_Center_for_Business_and_Human_Rights', 'https://vi.wikipedia.org/wiki/Khoa_h%E1%BB%8Dc_d%E1%BB%AF_li%E1%BB%87u', '#cite_note-25', '#cite_ref-:1_27-1', '/wiki/Category:Data_analysis', '#cite_ref-dsj02_16-0', '/wiki/International_Standard_Book_Number', '/wiki/List_of_datasets_for_machine-learning_research', '/wiki/K-nearest_neighbors_classification', '/wiki/Database', '/wiki/Open_science', 'https://de.wikipedia.org/wiki/Data_Science', '/wiki/Big_data', '/wiki/Generative_adversarial_network', 'http://www.nsf.gov/pubs/2005/nsb0540/', 'http://ur.umich.edu/9899/Nov09_98/4.htm', '/wiki/Special:MyContributions', '/wiki/Statistical_classification', 'https://es.wikipedia.org/wiki/Ciencia_de_datos', '/wiki/Computa

## Step 3: Use list comprehensions with conditions to clean the link list.

There are two types of links, absolute and relative. Absolute links have the full URL and begin with http while relative links begin with a forward slash (/) and point to an internal page within the wikipedia.org domain. Clean the respective types of URLs as follows.

- Absolute Links: Create a list of these and remove any that contain a percentage sign (%).
- Relative Links: Create a list of these, add the domain to the link so that you have the full URL, and remove any that contain a percentage sign (%).
- Combine the list of absolute and relative links and ensure there are no duplicates.

In [5]:
domain = 'http://wikipedia.org'

In [6]:
absolute = [i for i in links_3 if 'http' in i]
absolute_1 = [i for i in absolute if '%' not in i]
relative = [i for i in links_3 if i not in absolute]
relative_2 = [domain+i for i in relative]
relative_3 = [i for i in relative_2 if '%' not in i]
combine = list(set(absolute_1+relative_3))
combine_2 = [i.replace('http', 'https').replace('httpss','https') for i in combine]
combine_2

['https://wikipedia.org/wiki/Statistical_theory',
 'https://hbr.org/2012/10/data-scientist-the-sexiest-job-of-the-21st-century/',
 'https://wikipedia.org/wiki/National_Institutes_of_Health',
 'https://www.springer.com/book/9784431702085',
 'https://wikipedia.org/wiki/Deep_learning',
 'https://wikipedia.org#cite_note-NateSilver-26',
 'https://wikipedia.org#cite_ref-ics12_14-0',
 'https://wikipedia.org/wiki/Convolutional_neural_network',
 'https://wikipedia.org#cite_note-cfjwutk-10',
 'https://wikipedia.org#cite_note-32',
 'https://wikipedia.org/wiki/Help:Contents',
 'https://de.wikipedia.org/wiki/Data_Science',
 'https://wikipedia.org//www.worldcat.org/issn/0036-8075',
 'https://wikipedia.org/wiki/K-nearest_neighbors_algorithm',
 'https://wikipedia.org#cite_note-21',
 'https://wikipedia.org/wiki/Digital_object_identifier',
 'https://wikipedia.org#cite_note-Harvard-20',
 'https://wikipedia.org/wiki/Template_talk:Machine_learning_bar',
 'https://wikipedia.org#cite_ref-:2_28-7',
 'https://

## Step 4: Use the os library to create a folder called *wikipedia* and make that the current working directory.

In [7]:
import os

In [10]:
#'C:\Users\aquir\IronHack\datamex0120\module-1\lab-parallelization\your-code\wikipedia'
os.mkdir(r'C:\Users\aquir\IronHack\datamex0120\module-1\lab-parallelization\your-code\wikipedia')
os.chdir(r'C:\Users\aquir\IronHack\datamex0120\module-1\lab-parallelization\your-code\wikipedia')

## Step 5: Write a function called index_page that accepts a link and does the following.

- Tries to request the content of the page referenced by that link.
- Slugifies the filename using the `slugify` function from the [python-slugify](https://pypi.org/project/python-slugify/) library and adds a .html file extension.
    - If you don't already have the python-slugify library installed, you can pip install it as follows: `$ pip install python-slugify`.
    - To import the slugify function, you would do the following: `from slugify import slugify`.
    - You can then slugify a link as follows `slugify(link)`.
- Creates a file in the wikipedia folder using the slugified filename and writes the contents of the page to the file.
- If an exception occurs during the process above, just `pass`.

In [11]:
from slugify import slugify

In [12]:
def index_page(link):  
    try:
        html=requests.get(link).content
        slug=slugify(html)
        file=open('slugified','w')
        file=file.write(slug)
    except:
        pass

## Step 6: Sequentially loop through the list of links, running the index_page function each time.

Remember to include `%%time` at the beginning of the cell so that it measures the time it takes for the cell to run.

In [13]:
%%time
sequence = [index_page(i) for i in combine_2]

Wall time: 5min 12s


## Step 7: Perform the page indexing in parallel and note the difference in performance.

Remember to include `%%time` at the beginning of the cell so that it measures the time it takes for the cell to run.

In [14]:
import multiprocessing

In [ ]:
%%time
pool = multiprocessing.Pool()
result = pool.map(index_page(), combine_2)
pool.terminate()
pool.join()